In [82]:
import requests 
import os
import pandas as pd
import numpy as np 
import plotly.express as px

In [ ]:
#CREAR LA FUNCIÓN DE OBTENCIÓN DE PERSONAJES MEDIANTE EL USO DE API 

"""
   La función "obtener_datos_personajes" crea una lista vacía llamada personajes.
   Establece una variable i en 0.
   Mientras la condición sea verdadera, realiza una solicitud GET a la URL establecida en la variable base_url,
   pasando como parámetro el valor de la variable i más 1.
   La variable d recibe el resultado de la solicitud, el cual es un archivo JSON, y se almacena el valor de "results".
   Si el resultado es nulo, se rompe el bucle.
   En caso contrario, se extiende la lista personajes con el resultado de la solicitud y se incrementa la variable i en 1.
"""

In [71]:
def obtener_datos_personajes(base_url):
    personajes = []
    i = 0
    while True:
        url_con_paginacion = f"{base_url}?page={i+1}" # para las webs con más de una página
        respuesta = requests.get(url_con_paginacion) # Se solicita la url
        datos = respuesta.json() #para pasar a JSON
        resultados = datos.get('results')
        if not resultados:
            break
        personajes.extend(resultados)
        i += 1
    return personajes

base_url = "https://rickandmortyapi.com/api/character"
datos_personajes = obtener_datos_personajes(base_url)
df = pd.DataFrame(datos_personajes)


In [58]:
df.shape

(826, 12)

#### Guardas los datos en forma de csv

In [59]:
df.to_csv('personajes_rickymorty')

#### Limpiamos la columna origin dejando solo el nombre de donde nació (despues de 'name': y antes de url': '')

In [72]:
def primer_valor(diccionario): #La columna 'origin' está compues de diccionarios. Creamos una función que se quede con el valor de la primera clave de cada diccionario.
    if diccionario:
        return diccionario[list(diccionario.keys())[0]]
    return None

df['origin'] = df['origin'].apply(primer_valor)
df.head(3)

,id,name,status,species,type,gender,origin,location,image,episode,url,created
0,1,Rick Sanchez,Alive,Human,,Male,Earth (C-137),"{'name': 'Citadel of Ricks', 'url': 'https://r...",https://rickandmortyapi.com/api/character/avat...,"[https://rickandmortyapi.com/api/episode/1, ht...",https://rickandmortyapi.com/api/character/1,2017-11-04T18:48:46.250Z
1,2,Morty Smith,Alive,Human,,Male,unknown,"{'name': 'Citadel of Ricks', 'url': 'https://r...",https://rickandmortyapi.com/api/character/avat...,"[https://rickandmortyapi.com/api/episode/1, ht...",https://rickandmortyapi.com/api/character/2,2017-11-04T18:50:21.651Z
2,3,Summer Smith,Alive,Human,,Female,Earth (Replacement Dimension),"{'name': 'Earth (Replacement Dimension)', 'url...",https://rickandmortyapi.com/api/character/avat...,"[https://rickandmortyapi.com/api/episode/6, ht...",https://rickandmortyapi.com/api/character/3,2017-11-04T19:09:56.428Z


#### hago lo mismo con la columna location 

In [73]:
df['location'] = df['location'].apply(primer_valor)  # se puede aplicar la función creada antes.
df.head(3)

,id,name,status,species,type,gender,origin,location,image,episode,url,created
0,1,Rick Sanchez,Alive,Human,,Male,Earth (C-137),Citadel of Ricks,https://rickandmortyapi.com/api/character/avat...,"[https://rickandmortyapi.com/api/episode/1, ht...",https://rickandmortyapi.com/api/character/1,2017-11-04T18:48:46.250Z
1,2,Morty Smith,Alive,Human,,Male,unknown,Citadel of Ricks,https://rickandmortyapi.com/api/character/avat...,"[https://rickandmortyapi.com/api/episode/1, ht...",https://rickandmortyapi.com/api/character/2,2017-11-04T18:50:21.651Z
2,3,Summer Smith,Alive,Human,,Female,Earth (Replacement Dimension),Earth (Replacement Dimension),https://rickandmortyapi.com/api/character/avat...,"[https://rickandmortyapi.com/api/episode/6, ht...",https://rickandmortyapi.com/api/character/3,2017-11-04T19:09:56.428Z


#### elimino la columna image, url y created ya que no las vamos a usar en este ejercicio

In [79]:
df2 = df.drop(['image','url','created'], axis=1)
df2.head()

,id,name,status,species,type,gender,origin,location,episode
0,1,Rick Sanchez,Alive,Human,,Male,Earth (C-137),Citadel of Ricks,"[https://rickandmortyapi.com/api/episode/1, ht..."
1,2,Morty Smith,Alive,Human,,Male,unknown,Citadel of Ricks,"[https://rickandmortyapi.com/api/episode/1, ht..."
2,3,Summer Smith,Alive,Human,,Female,Earth (Replacement Dimension),Earth (Replacement Dimension),"[https://rickandmortyapi.com/api/episode/6, ht..."
3,4,Beth Smith,Alive,Human,,Female,Earth (Replacement Dimension),Earth (Replacement Dimension),"[https://rickandmortyapi.com/api/episode/6, ht..."
4,5,Jerry Smith,Alive,Human,,Male,Earth (Replacement Dimension),Earth (Replacement Dimension),"[https://rickandmortyapi.com/api/episode/6, ht..."


#### Proporción de géneros


In [94]:
generos = df['gender'].value_counts().reset_index()
fig = px.pie(generos, values='count', names='gender', 
             title = 'Proporción de géneros', 
             labels={'gender':'Género','count':'Cantidad'}, 
             template= 'plotly_dark')
fig.update_layout(width=800, height=600)
fig.show()

#### Cómo se distribuyen las especies

In [120]:
especies = df[['species','name']].value_counts().reset_index()
especies
px.treemap(especies, path=['species','name'], values='count', 
           height=700, title='Tamaño de las especies', 
           color_discrete_sequence = px.colors.qualitative.Dark2, 
           template= 'plotly_dark',
           labels={'labels':'Nombre','count':'Cantidad', 'parent':'Especie'})